# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this sql statement to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
```

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('lego.db')
cur = conn.cursor()

In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
table_names = pd.Series(cur.fetchall())
table_names

0    (product_details,)
1       (product_info,)
2    (product_pricing,)
3    (product_reviews,)
dtype: object

In [4]:
cur.execute("SELECT * FROM product_details")
details_df = pd.DataFrame(cur.fetchall())
details_df.columns = [i[0] for i in cur.description]
display(details_df.head())
details_df.info()

,prod_id,prod_desc,prod_long_desc,theme_name
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®
2,7280,blah,blahblah,City
3,7281,blah,blahblah,City
4,7499,blah,blahblah,City


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 4 columns):
prod_id           744 non-null int64
prod_desc         744 non-null object
prod_long_desc    744 non-null object
theme_name        744 non-null object
dtypes: int64(1), object(3)
memory usage: 23.3+ KB


In [5]:
cur.execute("SELECT * FROM product_info")
info_df = pd.DataFrame(cur.fetchall())
info_df.columns = [i[0] for i in cur.description]
display(info_df.head())
info_df.info()

,prod_id,ages,piece_count,set_name
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate
2,7280,5-12,2,Straight & Crossroad Plates
3,7281,5-12,2,T-Junction & Curved Road Plates
4,7499,5-12,24,Flexible and Straight Tracks


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 4 columns):
prod_id        744 non-null int64
ages           744 non-null object
piece_count    744 non-null int64
set_name       744 non-null object
dtypes: int64(2), object(2)
memory usage: 23.3+ KB


In [6]:
cur.execute("SELECT * FROM product_pricing")
pricing_df = pd.DataFrame(cur.fetchall())
pricing_df.columns = [i[0] for i in cur.description]
display(pricing_df.head())
pricing_df.info()

,prod_id,country,list_price
0,75823,US,29.99
1,75822,US,19.99
2,75821,US,12.99
3,21030,US,99.99
4,21035,US,79.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 3 columns):
prod_id       10870 non-null int64
country       10870 non-null object
list_price    10870 non-null object
dtypes: int64(1), object(2)
memory usage: 254.8+ KB


In [7]:
cur.execute("SELECT * FROM product_reviews")
reviews_df = pd.DataFrame(cur.fetchall())
reviews_df.columns = [i[0] for i in cur.description]
display(reviews_df.head())
reviews_df.info()

,prod_id,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3
2,7280,85.0,4.1,Very Easy,3.5,2.3
3,7281,40.0,4.1,Very Easy,3.9,2.8
4,7499,89.0,2.9,Easy,2.5,2.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 6 columns):
prod_id              744 non-null int64
num_reviews          655 non-null float64
play_star_rating     648 non-null float64
review_difficulty    744 non-null object
star_rating          655 non-null float64
val_star_rating      647 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 35.0+ KB


In [8]:
lego_merged = pd.merge(pricing_df, details_df)
lego_merged = pd.merge(lego_merged, info_df)
lego_merged = pd.merge(lego_merged, reviews_df)
display(lego_merged.head())
lego_merged.info()

,prod_id,country,list_price,prod_desc,prod_long_desc,theme_name,ages,piece_count,set_name,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,US,29.99,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,2.0,4.0,Average,4.5,4.0
1,75823,CA,31.192200000000003,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,2.0,4.0,Average,4.5,4.0
2,75822,US,19.99,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,2.0,4.0,Easy,5.0,4.0
3,75822,CA,19.4922,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,2.0,4.0,Easy,5.0,4.0
4,75821,US,12.99,blah,blahblah,Angry Birds™,6-12,74,Piggy Car Escape,11.0,4.3,Easy,4.3,4.1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
country              10870 non-null object
list_price           10870 non-null object
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


In [10]:
lego_merged.to_csv('Merged_Dataset.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!